# Basic Boolean Search in Documents

## Objective
Expand the simple term search functionality to include Boolean search capabilities. This will allow users to perform more complex queries by combining multiple search terms using Boolean operators.

## Problem Description
You must enhance the existing search engine from the previous exercise to support Boolean operators: AND, OR, and NOT. This will enable the retrieval of documents based on the logical relationships between multiple terms.

# Requirements
## Step 1: Update Data Preparation
Ensure that the documents are still loaded and preprocessed from the previous task. The data should be clean and ready for advanced querying.

In [1]:
import os
from pathlib import Path
import re
#from collections import defaultdict
from collections import Counter
import numpy as np

In [2]:
def buscar_titulos(directorio):
    archivos_titulos = []
    # Iterar sobre todos los archivos en el directorio
    for archivo in directorio.glob('*.txt'):
            # Abrir el archivo y buscar la palabra
            titulo=archivo.stem
            archivos_titulos.append(str(titulo))
    return archivos_titulos

In [3]:
def diccionario_libros(directorio):
    palabras_individuales = []
    for archivo in directorio.glob('*.txt'):
            with open((archivo), 'r', encoding='utf-8') as file:   
             contenido = file.read()
            # Limpiar el contenido: eliminar caracteres especiales y dividir en palabras
             palabras = re.findall(r'\b[A-Za-z0-9]+\b', contenido.lower())
             palabras_individuales.extend(palabras)      
    return palabras_individuales

In [4]:
directorio_a_buscar = Path("data")  # Ruta de la carpeta "data"
titulos_encontrados = buscar_titulos(directorio_a_buscar)


In [5]:
palabras_diccionario=diccionario_libros(directorio_a_buscar)
palabras_diccionario= set(palabras_diccionario)
palabras_ordenadas = sorted(palabras_diccionario)
print(palabras_ordenadas)

['0', '00', '000', '0000', '000000', '000002', '000007', '000012', '000012lt', '000012t', '000014t', '000025', '000045', '000198', '00037', '00045', '00055', '000814', '00097', '0009m', '000h', '001', '001024', '00107', '00117', '0011m', '001389', '0019m', '001v', '002', '00211', '0023m', '00248', '0025m', '0027m', '0029m', '003', '0031', '0035m', '0037m', '00385t', '0039m', '0041', '0041m', '0042', '0045m', '0051m', '0056m', '0057m', '0059m', '0065m', '0067', '0069m', '0071m', '0073m', '0075m', '0079m', '0081m', '008333', '0085m', '0087m', '009', '0091m', '0093m', '0097m', '0099m', '01', '0103m', '0105m', '0107m', '011', '0111', '0111m', '0113m', '0114', '0119m', '012', '0120m', '0121', '0122', '0123', '01231', '0123m', '0124', '0125m', '0127m', '013', '0131', '0133m', '0137m', '014', '0141', '0141m', '0145m', '0147m', '0148m', '015', '0151m', '0159m', '016', '0161m', '0165m', '017', '0173m', '0175m', '0177m', '018', '0183', '0185m', '0189m', '019', '0191m', '0193m', '0197m', '02', '0

## Step 2: Create an Inverted Index
Create an inverted index from the documents. This index maps each word to the set of document IDs in which that word appears. This facilitates word lookup in the search process.

In [6]:
filas=len(palabras_diccionario)
columnas=len(titulos_encontrados)
matriz = np.zeros((filas, columnas), dtype=bool)
print(matriz)

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [7]:

def palabras_por_libro(directorio,libro_nombre):
    palabras_individuales1 = []
    archivo = directorio / (libro_nombre + '.txt')
    # Verificar si el archivo existe
    if archivo.exists():
        with open(archivo, 'r', encoding='utf-8') as file:
            contenido = file.read()
            # Limpiar el contenido: eliminar caracteres especiales y dividir en palabras
            palabras = re.findall(r'\b[A-Za-z0-9]+\b', contenido.lower())
            palabras_individuales1.extend(palabras) 
            palabras_individuales1 = set(palabras_individuales1)
            return palabras_individuales1

In [8]:
 for j, titulo in enumerate(titulos_encontrados):
    palabras_por_libro_actual= palabras_por_libro(directorio_a_buscar,titulo)
    for i, palabra in enumerate(palabras_ordenadas):
        if palabra in palabras_por_libro_actual:
            matriz[i, j] = True

In [9]:
print(matriz)

[[ True False  True ... False False False]
 [ True False False ... False False False]
 [ True  True  True ...  True  True  True]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [10]:
diccionario_bi= dict(enumerate(palabras_ordenadas))
titulos_bi= dict(enumerate(titulos_encontrados))

## Step 3: Query Processing
Parse the Query: Implement a function to parse the input query to identify the terms and operators.
Search Documents: Based on the parsed query, implement the logic to retrieve and rank the documents according to the Boolean expressions.

In [11]:
palabra_a_buscar = input("Ingresa la palabra que quieres buscar: ") 

Ingresa la palabra que quieres buscar: zulu


In [12]:
indice = None
for key, value in diccionario_bi.items():
    if value == palabra_a_buscar:
        indice = key
        break     
if indice is not None:
    print("La palabra esta en la fila:", indice)
else:
    print("La palabra no hay en la matriz.")

La palabra esta en la fila: 184120


In [13]:
contador_columnas_totales=[]
for j in range(matriz.shape[1]):
    #print(j)
    indice=indice
    if matriz[indice, j] == 1:
        contador_columnas_totales.append(j)
print("Columnas donde esta la palabra:")
print(contador_columnas_totales)

Columnas donde esta la palabra:
[94]


In [14]:
palabra_encontrada_titulos=[]
for key, value in titulos_bi.items():
    if key in contador_columnas_totales:
        palabra_encontrada_titulos.append(value)
        continue    

## Step 4: Displaying Results
Output the Results: Display the documents that match the query criteria. Include functionalities to handle queries that result in no matching documents.

In [15]:

if palabra_encontrada_titulos:
    print("La palabra "+palabra_a_buscar+" se encontro en los siguientes archivos:")
    for archivo in palabra_encontrada_titulos:
        print(archivo)
else:
    print("La palabra "+palabra_a_buscar+" no se encontro en ningun archivo en el directorio especificado.")

La palabra zulu se encontro en los siguientes archivos:
Ulysses
